In [2]:
!pip install torch torchvision pandas matplotlib scikit-learn


In [22]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
import sys
sys.path.append('/content/drive/My Drive/QuadNet/modelInitialize')

In [7]:
print(os.listdir('/content/drive/My Drive/QuadNet'))

['basic_CNN_training.py', 'test.py', 'MQN-training all file together.py', 'README.md', 'modelInitialize.py', 'MQN.py', 'hyperparameter check.py', 'MQN-continues training.py', '.git', '__pycache__']


In [9]:
from google.colab import files
uploaded = files.upload()

Saving modelInitialize.py to modelInitialize.py


In [17]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import os
from modelInitialize import SmallQuadNet
import matplotlib.pyplot as plt

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [30]:


def calculate_distance(North,East,Down):

    distance = []
    for i in range(1,len(North)):
        distance.append(np.sqrt((North[i]-North[i-1])**2+(East[i]-East[i-1])**2+(Down[i]-Down[i-1])**2))
    return distance

# Create sequences for time-series data per GT
def create_sequences(features, target, window_size,timesteps):
    X, y = [], []
    max_index = len(target)
    for i in range(0, max_index, window_size // 2):#stride is half the window size
        if len(features[i:]) < window_size or i+timesteps >= max_index:
            break
        X.append(features[i:i + window_size])
        y.append(target[i+timesteps])
    return np.array(X), np.array(y)

# Define the train_and_eval function for continuous training
def train(model, optimizer, criterion,window_size,timesteps, imu_data, gt_data, iteration):

    # Extract relevant features (IMU data) and target (distance)
    imu_features = imu_data[["Acc_X", "Acc_Y", "Acc_Z", "Gyr_X", "Gyr_Y", "Gyr_Z"]].values
    North=gt_data["North"].values
    East=gt_data["East"].values
    Down=gt_data["Down"].values
    distance = calculate_distance(North,East,Down)

    # Normalize the features- consider batch normalization instead
    #scaler = StandardScaler()
    #imu_features_normalized = scaler.fit_transform(imu_features)

    X, y = create_sequences(imu_features, distance, window_size,timesteps)
    # Move X and y to the device (GPU if available)
    X, y = torch.tensor(X, dtype=torch.float32).to(device), torch.tensor(y ,dtype=torch.float32).to(device)

    # Training loop
    num_epochs =70
    batch_size = 64
    model.train()
    lambda_reg = 0.01

    # Initialize batch normalization layer and move it to the correct device.
    batch_norm = nn.BatchNorm1d(X.size(2)).to(device)

    for epoch in range(num_epochs):

        epoch_loss = 0
        for i in range(0, len(X), batch_size):
            inputs = X[i:i + batch_size]
            #Adjust input shape for BatchNorm1d: [batch_size, features, sequence_length]
            inputs = inputs.permute(0, 2, 1)  # Shape: [batch_size, features, sequence_length]

            # Batch normalization
            inputs_normalized = batch_norm(inputs)
            targets = y[i:i + batch_size]

            # Adjust input shape for Conv1D
            #inputs_normalized = inputs_normalized.permute(0, 2, 1)

            # Forward pass
            optimizer.zero_grad()
            outputs = model(inputs_normalized)
            mse_loss = criterion(outputs.squeeze(), targets.squeeze())#MSE loss
            # L2 Regularization (sum of squared weights)
            l2_reg = 0
            for param in model.parameters():
                l2_reg += torch.norm(param, p=2) ** 2

            # Combine MSE loss with regularization
            loss = mse_loss + lambda_reg * l2_reg
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

        scheduler.step(epoch_loss / len(X))
        print(f"Iteration: {iteration} | Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")
    return model



def test(model, criterion, window_size, timesteps, imu_data, gt_data, iteration):
    imu_features = imu_data[["Acc_X", "Acc_Y", "Acc_Z", "Gyr_X", "Gyr_Y", "Gyr_Z"]].values
    North=gt_data["North"].values
    East=gt_data["East"].values
    Down=gt_data["Down"].values
    distance = calculate_distance(North,East,Down)

    # Normalize the features
    scaler = StandardScaler()
    imu_features_normalized = scaler.fit_transform(imu_features)

    X, y = create_sequences(imu_features_normalized, distance, window_size, timesteps)
    # Move X and y to the device (GPU if available)
    X, y = torch.tensor(X, dtype=torch.float32).to(device), torch.tensor(y, dtype=torch.float32).to(device)

    # Initialize batch normalization layer and move it to the correct device.
    batch_norm = nn.BatchNorm1d(X.size(2)).to(device)

    # Evaluate the model
    model.eval()
    with torch.no_grad():
        X = X.permute(0, 2, 1)  # Adjust shape for testing
        predictions = model(X).squeeze()
        test_loss = criterion(predictions, y)
        print(f"Iteration: {iteration} | Test Loss: {test_loss.item():.4f}")

        # Calculate Mean Absolute Error (MAE) as accuracy metric
        mae = torch.mean(torch.abs(predictions - y))
        print(f"Iteration: {iteration} | Test MAE: {mae.item():.4f}")

        # Calculate accuracy (percentage of predictions within a certain threshold)
        threshold = 0.1  # Define a threshold for accuracy
        accuracy = torch.mean((torch.abs(predictions - y) < threshold).float()) * 100
        print(f"Iteration: {iteration} | Test Accuracy: {accuracy.item():.2f}%")

        # Append loss, MAE, and accuracy to lists for plotting
        losses.append(test_loss.item())
        maes.append(mae.item())
        accuracies.append(accuracy.item())
    return model

# Set up directories
losses, maes, iterations,accuracies = [], [], [],[]

parent_folder = "/content/gdrive/MyDrive/Quadrotor-Dead-Reckoning-with-Multiple-Inertial-Sensors/Horizontal"
print(os.listdir(parent_folder))
# Initialize the PyTorch model, loss function, and optimizer
n_features = 6  # IMU features (fixed for all folders)
model = SmallQuadNet(Input=n_features, imu_window_size=120)
model = model.to(device)
criterion = nn.MSELoss()
""""
# Define a regularization term (L2 regularization)
def regularized_loss(output, target, model, lambda_l2=0.01):
    mse_loss = criterion(output, target)
    l2_reg = torch.tensor(0., requires_grad=True)
    for param in model.parameters():
        l2_reg = l2_reg + torch.norm(param, 2)
    return mse_loss + lambda_l2 * l2_reg
"""
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.7, patience=4, verbose=True)
# Train and evaluate across all folders continuously
iteration=0
for i in range(1, 27):

    iteration += 1  # Increment iteration after loading checkpoint
    folder_name = f"path_{i}"
    folder_path = os.path.join(parent_folder, folder_name)

    if os.path.isdir(folder_path):
        gt_file = os.path.join(folder_path, "GT.csv")
        imu_file = os.path.join(folder_path, "IMU_1.csv")

        if os.path.exists(gt_file) and os.path.exists(imu_file):
            gt_data = pd.read_csv(gt_file)
            imu_data = pd.read_csv(imu_file)
            #gt_data, imu_data = gt_data.to(device), imu_data.to(device)
            window_size = 120
            timesteps=int(120/(len(imu_data) / len(gt_data)))
            print(f"Training on folder {folder_path} (Iteration {iteration})")
            model = train(model, optimizer, criterion,window_size,timesteps, imu_data, gt_data, iteration)

        else:
            print(f"Missing data in folder {folder_path}, skipping.")
    else:
        print(f"Folder {folder_path} does not exist, skipping.")

num_test=1
for i in range(27, 28):

    iteration += 1  # Increment iteration after loading checkpoint
    folder_name = f"path_{i}"
    folder_path = os.path.join(parent_folder, folder_name)

    if os.path.isdir(folder_path):
        gt_file = os.path.join(folder_path, "GT.csv")
        imu_file = os.path.join(folder_path, "IMU_1.csv")

        if os.path.exists(gt_file) and os.path.exists(imu_file):
            gt_data = pd.read_csv(gt_file)
            imu_data = pd.read_csv(imu_file)
            #gt_data, imu_data = gt_data.to(device), imu_data.to(device)
            window_size = 120
            timesteps=int(120/(len(imu_data) / len(gt_data)))
            print(f"test on folder {folder_path} (Iteration {iteration})")
            model = test(model, criterion,window_size,timesteps, imu_data, gt_data, iteration)
            print("accuracy",accuracies[num_test-1])
            print("mae",maes[num_test-1])
            print("loss",losses[num_test-1])
        else:
            print(f"Missing data in folder {folder_path}, skipping.")
    else:
        print(f"Folder {folder_path} does not exist, skipping.")



['guide.txt', 'path_9', 'path_8', 'path_7', 'path_4', 'path_5', 'path_6', 'path_26', 'path_27', 'path_3', 'path_25', 'path_24', 'path_23', 'path_22', 'path_2', 'path_21', 'path_20', 'path_19', 'path_17', 'path_18', 'path_16', 'path_15', 'path_14', 'path_13', 'path_11', 'path_12', 'path_10', 'path_1']


/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Training on folder /content/gdrive/MyDrive/Quadrotor-Dead-Reckoning-with-Multiple-Inertial-Sensors/Horizontal/path_1 (Iteration 1)
Iteration: 1 | Epoch [1/70], Loss: 11.4338
Iteration: 1 | Epoch [2/70], Loss: 10.1163
Iteration: 1 | Epoch [3/70], Loss: 9.7221
Iteration: 1 | Epoch [4/70], Loss: 9.0283
Iteration: 1 | Epoch [5/70], Loss: 8.8292
Iteration: 1 | Epoch [6/70], Loss: 8.7038
Iteration: 1 | Epoch [7/70], Loss: 8.6065
Iteration: 1 | Epoch [8/70], Loss: 8.4622
Iteration: 1 | Epoch [9/70], Loss: 8.2480
Iteration: 1 | Epoch [10/70], Loss: 8.0615
Iteration: 1 | Epoch [11/70], Loss: 7.8037
Iteration: 1 | Epoch [12/70], Loss: 7.5845
Iteration: 1 | Epoch [13/70], Loss: 7.3877
Iteration: 1 | Epoch [14/70], Loss: 7.4391
Iteration: 1 | Epoch [15/70], Loss: 7.0770
Iteration: 1 | Epoch [16/70], Loss: 6.9156
Iteration: 1 | Epoch [17/70], Loss: 6.7893
Iteration: 1 | Epoch [18/70], Loss: 6.6892
Iteration: 1 | Epoch [19/70], Loss: 6.5819
Iteration: 1 | Epoch [20/70], Loss: 6.4556
Iteration: 1 | E